In [10]:
import tensorflow as tf
from tensorflow import keras
import kerastuner
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import os
import json
import datetime

plt.rcParams["figure.figsize"] = (20, 5)

physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

In [3]:
dataset_name = "SEG_2"

In [4]:
dataset = np.genfromtxt("data/{}_train_set.csv".format(dataset_name), delimiter="\n", dtype=np.float32) #np.int64
dataset

array([0., 0., 0., ..., 1., 3., 1.], dtype=float32)

In [5]:
word_index = np.genfromtxt("data/word_index.csv", delimiter="\n", dtype=np.int64)
vocab_size = len(word_index)
vocab_size

14882

In [262]:
param_list = dict()

param_list["PAST_HISTORY"] = 64
param_list["FUTURE_TARGET"] = 8
param_list["BATCH_SIZE"] = 128
param_list["EPOCHS"] = 100
param_list["BUFFER_SIZE"] = 200000
param_list["VOCAB_SIZE"] = vocab_size
param_list["LEARNING_RATE"] = 0.01
param_list["NUM_1_NEURONS"] = 177
param_list["NUM_2_NEURONS"] = 177
param_list["DROPOUT_1"] = 0.1
param_list["DROPOUT_2"] = 0.2

In [263]:
def generate_timeseries(dataset, start_index, end_index, history_size, target_size):
    data = []
    labels = []

    start_index = start_index + history_size
    if end_index is None:
        end_index = len(dataset) - target_size

    for i in range(start_index, end_index):
        indices = range(i-history_size, i)
        # Reshape data from (history_size,) to (history_size, n_feature)
        data.append(np.reshape(dataset[indices], (history_size, 1)))
        #data.append(dataset[indices])
        labels.append(np.reshape(dataset[i:i+target_size], (target_size, 1)))
        #labels.append(dataset[i:i+target_size])
    return np.array(data), np.array(labels)

In [264]:
x_train, y_train = generate_timeseries(dataset, 0, None, param_list["PAST_HISTORY"], param_list["FUTURE_TARGET"])
x_train.shape, y_train.shape

((14810, 64, 1), (14810, 8, 1))

In [209]:
from tensorflow.keras.layers import TimeDistributed, Dense, Conv1D, MaxPooling1D, Bidirectional, LSTM, Flatten, Reshape, Lambda

In [292]:
model_1 = keras.models.Sequential()
model_1.add(Conv1D(filters=32, kernel_size=3, padding="causal", activation='relu'))
model_1.add(MaxPooling1D(pool_size=2))
#model.add(Flatten())
#model.add(Lambda(lambda x:tf.expand_dims(x,axis=-1),input_shape=[None]))
model_1.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(param_list["NUM_1_NEURONS"])))
model_1.add(keras.layers.Dropout(param_list["DROPOUT_2"]))
model_1.add(keras.layers.RepeatVector(param_list["FUTURE_TARGET"]))
model_1.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(param_list["NUM_2_NEURONS"], return_sequences=True)))
model_1.add(keras.layers.Dropout(param_list["DROPOUT_2"]))
model_1.add(keras.layers.TimeDistributed(keras.layers.Dense(param_list["VOCAB_SIZE"], activation='softmax')))

model_1.compile(optimizer=keras.optimizers.Adam(param_list["LEARNING_RATE"]), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [293]:
result_1 = model_1.predict(x_train[10000].reshape(1, -1, 1))
result_1.shape

(1, 8, 14882)

In [294]:
model_1_history = model_1.fit(x_train, y_train, batch_size=param_list["BATCH_SIZE"], validation_split=0.2, epochs=param_list["EPOCHS"])

0s - loss: 0.3552 - accuracy: 0.8987/93 [===========================>..] - ETA: 0s - loss: 0.3556 - accuracy: 0.8989/93 [===========================>..] - ETA: 0s - loss: 0.3569 - accuracy: 0.8991/93 [============================>.] - ETA: 0s - loss: 0.3563 - accuracy: 0.8993/93 [==============================] - ETA: 0s - loss: 0.3562 - accuracy: 0.8993/93 [==============================] - 4s 38ms/step - loss: 0.3562 - accuracy: 0.8961 - val_loss: 0.5398 - val_accuracy: 0.8121
Epoch 96/100
 1/93 [..............................] - ETA: 0s - loss: 0.3599 - accuracy: 0.88 3/93 [..............................] - ETA: 2s - loss: 0.3478 - accuracy: 0.89 5/93 [>.............................] - ETA: 2s - loss: 0.3753 - accuracy: 0.88 7/93 [=>............................] - ETA: 2s - loss: 0.3714 - accuracy: 0.89 9/93 [=>............................] - ETA: 2s - loss: 0.3728 - accuracy: 0.8811/93 [==>...........................] - ETA: 2s - loss: 0.3693 - accuracy: 0.8813/93 [===>............

In [295]:
timestamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
'''
log_dir = "logs/fit/" + timestamp
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
'''
version_dir = "version/" + timestamp 

os.makedirs(version_dir)
timestamp

'20200909-143624'

In [296]:
model_1.save("version/{}/model.h5".format(timestamp))